In [29]:


def choose_sources(data, sources, allowedUse):

    sources = sources.split(", ")
    
    #find start and end point of analysis
    start = 0
    end = 10**100

    for x in range(len(sources)):
        s = data[sources[x]]
        _start = data[(s > 0)].index[0]
        if _start > start:
            start = _start        
        _end = data[(s > 0)].index[-1]
        if _end < end:
            end = _end

    #collect sources in a single pool
    source = 0
    for x in range(len(sources)):
        source = source + data[sources[x]]

    waterAvailable = []

    for x in range(0, len(source)):
        if x < start or x > end:
            waterAvailable.append(0)
        else:
            waterAvailable.append(source[x])
    
    _waterAvailable = []

    for x in range(len(waterAvailable)):
        _waterAvailable.append(waterAvailable[x] * allowedUse)
    
    return start, end, _waterAvailable

def WaterBalance(data, waterAvailable, cf):
    waterNeed = []

    for x in range(len(data)):
        if x < start or x > end:
            waterNeed.append(0)
        else:
            mWh = 24 * (cf / 100) * generationCapacity;
            waterNeed.append(waterToX * mWh / retentionFactor)

    generalBalance = []
    for x in range(len(data)):
        generalBalance.append(waterAvailable[x] - waterNeed[x])

    return waterNeed[start], generalBalance

def ReservoirSize(data, balance):
    #assume we start with fulll reservoir
    reservoirVolume = np.zeros(len(data))
    reservoirCapacity = 0 #small industrial water tank size
    reservoirVolume[start-1] = reservoirCapacity 
    reservoirRatio = []
    
    isSatisfied = False

    #reservoir empty outside of active time period
    for x in range(len(data)):
        if x < start or x > end:
            reservoirVolume[x] = 0

    #increase reservoir capacity until saftely limit is kept satisfied        
    while True:
        if isSatisfied == True:    
            break
        
        reservoirCapacity = reservoirCapacity + 100
        for x in range(start, len(data)):
                reservoirVolume[x] = reservoirVolume[x-1] + balance[x] 

                if reservoirVolume[x] > reservoirCapacity:
                    reservoirVolume[x] = reservoirCapacity

                if (reservoirVolume[x]/reservoirCapacity)*100 < buffersize:
                    #print("under safety limit")
                    break

                if x == end:
                    print(reservoirCapacity)
                    isSatisfied = True
                    break
                
                if reservoirCapacity > 10000:
                    reservoirCapacity = -1
                    
    #for x in range(len(data)):
    #    reservoirRatio.append((reservoirVolume[x]/reservoirCapacity)*100)
    
    return reservoirCapacity, reservoirVolume  




In [136]:
import pandas as pd
import numpy as np
from itertools import combinations
from itertools import permutations


#todo: fix smooth rainfall missing entries 
start = 366
end = 911

class Source: 
    def __init__(self, name, data, is_daily, s_type, capex, opex, rententate):

        self.name = name
        self.data = data      
        self.monthly_data = []     
        self.is_daily = is_daily
        self.s_type = s_type
        self.capex = capex
        self.opex = opex
        self.rententate = rententate
    
    def update(self, new_data):
        self.monthly_data = self.data 
        self.data = new_data
        self.is_daily = True
        
class SP2X:
    def __init__(self, gen_cap, water_2_x, retention_fac, cap_fac, sources): 

        self.gen_cap = gen_cap
        self.water_2_x = water_2_x
        self.retention_fac = retention_fac
        self.cap_fac = cap_fac
        self.sources = sources
        self.source_cap = 3
        self.buffer = 30 
        self.water_need = self.p2x_need()          
        
    def p2x_need(self):
        need = []     
        for _ in range(len(self.sources[0].data)):
            mWh = 24 * (self.cap_fac / 100) * self.gen_cap;
            need.append(self.water_2_x * mWh / self.retention_fac)
        return need

    def granulate(self, rainfall):
        for s in self.sources:
            if s.is_daily == False:    
                new_data = []
            for d in range(len(s.data)):
                if d == 0: 
                    return new_data.append(0)
                else:
                    return new_data.append(35.5728 * rainfall +  0.0253 * data + 53.0302 * smooth(rainfall)) 
            s.update(new_data)
    
    def smooth(data):    
        avg_data = []
        n=5
        for x in range(n):
            avg_data.append(data[x])
        for x in range(n, len(data)):
            avg = 0
            for y in range(n):
                avg += data[x-y] 
            avg_data.append(avg / n)
        return avg_data
    
    class PooledSource:
        def __init__(self, source_list, water_need, buffer):
            
            s0 = source_list.pop()           
            self.amounts = s0.data
            self.names = s0.name 
            self.reservoir_size = None
            self.reservoir_history = None
            self.water_need = water_need 
            self.buffer = buffer
            
            while source_list:
                s = source_list.pop()
                self.amounts += s.data
                self.names += " + " + s.name
            
            self.reservoir_size, self.reservoir_history = self.estimate_reservoir()
        
        def estimate_reservoir(self):
            balance = self.water_balance()
            
            reservoirVolume = np.zeros(len(self.amounts))
            reservoirCapacity = 0 
            reservoirVolume[start-1] = reservoirCapacity 
            reservoirRatio = []
            isSatisfied = False
            
            while not isSatisfied:
                reservoirCapacity += 100

                for x in range(start, len(self.amounts)):
                    rv = reservoirVolume
                    rv[x] = min(rv[x-1] + balance[x], reservoirCapacity)

                    if rv[x] > reservoirCapacity: 
                        rv[x] = reservoirCapacity
                    if (rv[x] / reservoirCapacity) * 100 < self.buffer: 
                        break
                    if x == end: 
                        isSatisfied = True; break
                    if reservoirCapacity > 10000: reservoirCapacity = -1
            
            return reservoirCapacity, reservoirVolume  
            
        def water_balance(self):
            balance = []
            #TODO: change from zero to none (or equivilent), after fixing missing values as zero 
            for x in range(len(data)):
                if self.amounts[x] == 0:
                    balance.append(0)
                else:
                    balance.append(self.amounts[x] - self.water_need[x])
            return balance
        
    def compare(self, pools):
        best_pool = None
        while pools:
            p = pools.pop()
                                  #reservoir size > reservoir size
            if best_pool is None or p.amounts[500] > best_pool.amounts[500]:
                best_pool = p
        return best_pool

    def analysis(self):
        recs = []
        #iterations for all combinations of sources, using 1 up to souce_cap number of sources
        for i in range(1, p2x.source_cap + 1):
            pools = []
            for x in combinations(p2x.sources, i):
                #print([y for y in x])
                p = self.PooledSource([y for y in x], self.water_need, self.buffer)
                pools.append(p)
            recs.append(self.compare(pools))
        
        recs.reverse()
        for n, rec in enumerate(range(len(recs))):
            r = recs.pop()             
            s = " source, " if n == 0 else " sources, "                
            print("For " + str(n+1) + s + r.names + " is recommended")
            
            print(r.reservoir_size)
            

In [138]:
data = pd.read_csv(r'C:\Users\Lucas\Desktop\Arbejde\sekundavand\Data\preproccessing\monthVols\MonthData.csv', sep=';')

p1 = Source("Bruns Nord", data["P1MonthVol"], False, "floodwater", 0, 0, [])
p2 = Source("Normark", data["P2MonthVol"], False, "floodwater", 0, 0, [])
p3 = Source("Sandholm", data["P3MonthVol"], False, "floodwater", 0, 0, [])
p4 = Source("Hovsøre", data["P4MonthVol"], False, "floodwater", 0, 0, [])
p5 = Source("Plet Enge", data["P5MonthVol"], False, "floodwater", 0, 0, [])

p2x = SP2X(500, 0.2, 0.65, 66, [p1, p2, p3, p4, p5])
p2x.granulate(data["Rainfall"])

p2x.analysis()




For 1 source, Sandholm is recommended
100
For 2 sources, Plet Enge + Hovsøre is recommended
100
For 3 sources, Plet Enge + Hovsøre + Sandholm is recommended
100
